# Hurst Estimation

### Estimate method 1

In [2]:
import matplotlib.pyplot
import pandas as pd
import numpy as np
import math

In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()
# eng.addpath(r'C:/Users\Sacha\Documents\University\Yr 4, Sem 2\ENGG4802\code\hurst estimators\hurst estimator', nargout=0)
eng.addpath(r'hurst_estimators\hurst estimator', nargout=0)

In [3]:
data = pd.read_csv('data/filter_data.csv')
hurst_estimates = pd.read_csv('data/hurst_estimates_all.csv', index_col='Unnamed: 0')

## Calculate Hurst Exponent

In [4]:
HMethods = {'Agr Var.'   : eng.aggvar,
            'Peng'       : eng.peng, 
            'Higuchi'    : eng.higuchi, 
            'Abs val.'   : eng.absval, 
            'RS'         : eng.RS, 
            'Box per.'   : eng.boxper, 
            'Diff var.'  : eng.diffvar}

In [ ]:
hurst_estimates = pd.DataFrame(data=np.zeros((len(data.columns), 7)), columns=HMethods.keys(), index=data.columns)

for col_name in data.columns:
    mData = matlab.double(data[col_name].tolist())
    
    for h_name, h_func in HMethods.items():
        print(col_name, h_name, end=": ")
        hurst_estimates[h_name][col_name] = h_func(mData, 0)
        print(hurst_estimates[h_name][col_name])
        
    print()

window_1_0 Agr Var.: 0.8998373780928763
window_1_0 Peng: 1.1788145950992999
window_1_0 Higuchi: 0.8971471166822471
window_1_0 Abs val.: 0.8991657007189308
window_1_0 RS: 0.6420391348578887
window_1_0 Box per.: 1.0848706578159155
window_1_0 Diff var.: 1.2080842121878403

window_1_1 Agr Var.: 0.8574754662395951
window_1_1 Peng: 

In [9]:
hurst_estimates.head()

,Agr Var.,Peng,Higuchi,Abs val.,RS,Box per.,Diff var.
window_1_0,0.899837,1.178815,0.897147,0.899166,0.642039,1.084871,1.208084
window_1_1,0.857475,1.092218,0.878807,0.870051,0.622270,1.056530,1.101125
window_1_2,0.846924,1.055944,0.866159,0.860166,0.641788,1.009795,1.079708
window_1_3,0.875449,1.116465,0.879919,0.881300,0.664163,1.036084,1.172933
window_1_4,0.823455,0.972632,0.839136,0.843857,0.595073,0.953637,1.024767


In [8]:
# Check max values for each method
hurst_estimates.max(axis=0)

Agr Var.     0.919577
Peng         1.334193
Higuchi      0.917993
Abs val.     0.915664
RS           0.697658
Box per.     1.765978
Diff var.    1.696649
dtype: float64

In [11]:
# Save all methods results
hurst_estimates.to_csv('data/hurst_estimates_all.csv')

## Use only the reasonable estimates

In [6]:
# Aggregate Variance, Higuchi and Absolute Value methods were chosen due to their reasonable values ~ 0.9 that arn't
# above 1 and are all similar.
confident_hurst_estimates = hurst_estimates[['Agr Var.', 'Higuchi', 'Abs val.']].copy()

In [7]:
confident_hurst_estimates['avg'] = confident_hurst_estimates.mean(axis=1)
confident_hurst_estimates['std'] = confident_hurst_estimates.std(axis=1)

In [9]:
confident_hurst_estimates.head()

,Agr Var.,Higuchi,Abs val.,avg,std
window_1_0,0.899837,0.897147,0.899166,0.898717,0.001143
window_1_1,0.857475,0.878807,0.870051,0.868778,0.008755
window_1_2,0.846924,0.866159,0.860166,0.857749,0.008036
window_1_3,0.875449,0.879919,0.881300,0.878890,0.002497
window_1_4,0.823455,0.839136,0.843857,0.835483,0.008721


In [18]:
confident_hurst_estimates.to_csv('data/hurst_estimates.csv')